# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [ ]:
!pip install -qU openai

In [1]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [2]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [3]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "ML & DL Assistant" # @param {type: "string"}
instructions = '''You are an expert assistant answering technical questions on machine learning and deep learning subject.
Ensure that your response is unbiased and generic, you will be 'AWARDED' for giving really good clarity and correct answers.''' # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [4]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [5]:
print(assistant)

Assistant(id='asst_MapOlLsMbKqSNbRqi2fzFWMK', created_at=1713061548, description=None, file_ids=[], instructions="You are an expert assistant answering technical questions on machine learning and deep learning subject.\nEnsure that your response is unbiased and generic, you will be 'AWARDED' for giving really good clarity and correct answers.", metadata={}, model='gpt-3.5-turbo', name='ML & DL Assistant', object='assistant', tools=[])


There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [6]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [7]:
thread

Thread(id='thread_i0fXOv17V711RAdnpxdBL4W7', created_at=1713061554, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [8]:
thread.id

'thread_i0fXOv17V711RAdnpxdBL4W7'

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is Deep Learning - DL?"
)

Again, let's examine our `message` object!

In [10]:
message

Message(id='msg_MmxTeK7WTHulI0hvPonbnnAA', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is Deep Learning - DL?'), type='text')], created_at=1713061560, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_i0fXOv17V711RAdnpxdBL4W7')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [11]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = '''Think and give only explanation or code or links for resources or steps for project, for the questions asked along with response.
##EXAMPLES##
If users asks 'explain neural networks', your response should be with an overview of neural networks, discussing how they are computational models inspired by the human brain that are used to recognize patterns and solve complex problems in machine learning.
If users ask 'code convolutional neural network', your response should contain example of the code necessary to create a CNN.
If users ask 'resource gradient descent', your response should offer links to tutorials, video lectures, or articles that explain gradient descent, which is an optimization algorithm used to minimize a function by iteratively moving in the direction of steepest descent.
If users ask 'project sentiment analysis', your response should discuss the steps involved in creating a sentiment analysis model, such as data collection, preprocessing, model selection, training, and evaluation, and potentially offer advice on best practices or methodologies to consider.
''' # @param {type: "string"}

Let's run our Thread!

In [12]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [14]:
print(assistant.id)
print(thread.id)
print(message.id)

asst_MapOlLsMbKqSNbRqi2fzFWMK
thread_i0fXOv17V711RAdnpxdBL4W7
msg_MmxTeK7WTHulI0hvPonbnnAA


In [16]:
run

Run(id='run_4G5mZzo13ma3r5aQA9yRmM7F', assistant_id='asst_MapOlLsMbKqSNbRqi2fzFWMK', cancelled_at=None, completed_at=None, created_at=1713061683, expires_at=1713062283, failed_at=None, file_ids=[], instructions="You are an expert assistant answering technical questions on machine learning and deep learning subject.\nEnsure that your response is unbiased and generic, you will be 'AWARDED' for giving really good clarity and correct answers. Think and give only explanation or code or links for resources or steps for project, for the questions asked along with response.\n##EXAMPLES##\nIf users asks 'explain neural networks', your response should be with an overview of neural networks, discussing how they are computational models inspired by the human brain that are used to recognize patterns and solve complex problems in machine learning.\nIf users ask 'code convolutional neural network', your response should contain example of the code necessary to create a CNN.\nIf users ask 'resource gr

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [20]:
run.status

'completed'

In [18]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [19]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [22]:
print(assistant.id)
print(thread.id)
print(message.id)
print(run.id)

asst_MapOlLsMbKqSNbRqi2fzFWMK
thread_i0fXOv17V711RAdnpxdBL4W7
msg_MmxTeK7WTHulI0hvPonbnnAA
run_4G5mZzo13ma3r5aQA9yRmM7F


In [21]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

messages

SyncCursorPage[Message](data=[Message(id='msg_Ky5c8n1HWWgKES8y4sT8g1Rp', assistant_id='asst_MapOlLsMbKqSNbRqi2fzFWMK', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Deep Learning (DL) is a subfield of machine learning that focuses on teaching computers to learn and make decisions by simulating the human brain's neural networks. It involves the use of deep neural networks, which are composed of multiple layers of interconnected nodes (neurons) that work together to process data and extract relevant features. DL is particularly suited for handling complex tasks such as image and speech recognition, natural language processing, and other types of pattern recognition applications. By learning from large amounts of labeled data, deep learning models can automatically discover intricate patterns and relationships in the data, leading to highly accurate predictions and insights."), type='text')], created_at=1713061683, file_ids=[], incomplete_at=None, incomplet

In [36]:
print(messages.data)
print(len(messages.data))
messages.data[-1]

[Message(id='msg_Ky5c8n1HWWgKES8y4sT8g1Rp', assistant_id='asst_MapOlLsMbKqSNbRqi2fzFWMK', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Deep Learning (DL) is a subfield of machine learning that focuses on teaching computers to learn and make decisions by simulating the human brain's neural networks. It involves the use of deep neural networks, which are composed of multiple layers of interconnected nodes (neurons) that work together to process data and extract relevant features. DL is particularly suited for handling complex tasks such as image and speech recognition, natural language processing, and other types of pattern recognition applications. By learning from large amounts of labeled data, deep learning models can automatically discover intricate patterns and relationships in the data, leading to highly accurate predictions and insights."), type='text')], created_at=1713061683, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, 

Message(id='msg_MmxTeK7WTHulI0hvPonbnnAA', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is Deep Learning - DL?'), type='text')], created_at=1713061560, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_i0fXOv17V711RAdnpxdBL4W7')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [37]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [43]:
file_reference = client.files.create(
  file=open("frankenstein.html", "rb"),
  purpose='assistants'
)

In [44]:
file_reference

FileObject(id='file-U9s7r4kC9qiY6PfAGrtsZRWw', bytes=1218, created_at=1713069003, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None)

In [45]:
file_reference = client.files.create(
  file=open('/Users/nithin.kamavaram/Desktop/learning/AI_MAKERSPACE/Resources/week2/session4/2104.05314.pdf', "rb"),
  purpose='assistants'
)

In [46]:
file_reference

FileObject(id='file-RiqjOiU8HHACTiJMyYFfWkrU', bytes=406660, created_at=1713069007, filename='2104.05314.pdf', object='file', purpose='assistants', status='processed', status_details=None)

Let's look at what our `file_reference` contains!

In [47]:
file_reference

FileObject(id='file-RiqjOiU8HHACTiJMyYFfWkrU', bytes=406660, created_at=1713069007, filename='2104.05314.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [87]:
client.files.list().data

[FileObject(id='file-RiqjOiU8HHACTiJMyYFfWkrU', bytes=406660, created_at=1713069007, filename='2104.05314.pdf', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-U9s7r4kC9qiY6PfAGrtsZRWw', bytes=1218, created_at=1713069003, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-WhUMv6t7gcHdMDYbxmTNp0Zv', bytes=406660, created_at=1713068962, filename='2104.05314.pdf', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-mPKBxGbGskWHA58taJ3QMvXi', bytes=1218, created_at=1713068656, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None)]

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [56]:
file_reference.id

'file-RiqjOiU8HHACTiJMyYFfWkrU'

In [59]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)
assistant

Assistant(id='asst_3FIxHxogEnep6NAnJw2vbukr', created_at=1713104249, description=None, file_ids=['file-RiqjOiU8HHACTiJMyYFfWkrU'], instructions="You are an expert assistant answering technical questions on machine learning and deep learning subject.\nEnsure that your response is unbiased and generic, you will be 'AWARDED' for giving really good clarity and correct answers.", metadata={}, model='gpt-3.5-turbo', name='ML & DL Assistant+ Retrieval', object='assistant', tools=[RetrievalTool(type='retrieval')])

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [60]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is ML?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress


Let's look at the final result!

In [63]:
print(messages.data)
print(len(messages.data))
messages.data[-2]

[Message(id='msg_bvhYbHCenLxUhUAPUHklBDu7', assistant_id='asst_3FIxHxogEnep6NAnJw2vbukr', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Machine learning (ML) is a field that focuses on enabling computers to learn from data and improve their performance on specific tasks without being explicitly programmed. In essence, ML algorithms learn from example data to automate the process of building analytical models. This capacity to learn from training data enables computers to identify hidden insights and complex patterns without human intervention. ML has been applied successfully in various domains such as fraud detection, credit scoring, image and speech recognition, natural language processing, and many more. \n\nThere are three main types of machine learning:\n\n1. Supervised learning: In supervised learning, the algorithm learns patterns from labeled training data, where the input data is paired with the correct output. It is commonly used in application

Message(id='msg_bvhYbHCenLxUhUAPUHklBDu7', assistant_id='asst_3FIxHxogEnep6NAnJw2vbukr', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Machine learning (ML) is a field that focuses on enabling computers to learn from data and improve their performance on specific tasks without being explicitly programmed. In essence, ML algorithms learn from example data to automate the process of building analytical models. This capacity to learn from training data enables computers to identify hidden insights and complex patterns without human intervention. ML has been applied successfully in various domains such as fraud detection, credit scoring, image and speech recognition, natural language processing, and many more. \n\nThere are three main types of machine learning:\n\n1. Supervised learning: In supervised learning, the algorithm learns patterns from labeled training data, where the input data is paired with the correct output. It is commonly used in applications

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [64]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

In [65]:
file_deletion_status

FileDeleteResponse(id='file-RiqjOiU8HHACTiJMyYFfWkrU', deleted=True, object='assistant.file.deleted')

In [219]:
# check assisstants, assisstants files

my_assistants = client.beta.assistants.list(
    order="desc",
)
print(my_assistants.data)
print(len(my_assistants.data))

[Assistant(id='asst_McpTkpcakxbTjjPldMRudLFG', created_at=1713111965, description=None, file_ids=[], instructions="You are an expert assistant answering technical questions on machine learning and deep learning subject.\nEnsure that your response is unbiased and generic, you will be 'AWARDED' for giving really good clarity and correct answers.", metadata={}, model='gpt-3.5-turbo', name='ML & DL Assistant + All Tools', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter'), RetrievalTool(type='retrieval'), FunctionTool(function=FunctionDefinition(name='duckduckgo_search', description='Answer non-technical questions. ', parameters={'type': 'object', 'properties': {'query': {'type:': 'string', 'description': "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"}}, 'required': ['query']}), type='function')]), Assistant(id='asst_qaPLg2lbWOW3ROjVCq8yUYu0', created_at=1713108056, description=None, file_ids=[], instructions="You are an e

In [220]:

'''assistant_files = client.beta.assistants.files.list(
  assistant_id="asst_qaPLg2lbWOW3ROjVCq8yUYu0"
)
print(assistant_files)'''

'assistant_files = client.beta.assistants.files.list(\n  assistant_id="asst_qaPLg2lbWOW3ROjVCq8yUYu0"\n)\nprint(assistant_files)'

In [221]:
'''response = client.beta.assistants.delete("asst_nlEfM5YuLVwTooMfdfuU2lYB")
print(response)'''


'response = client.beta.assistants.delete("asst_nlEfM5YuLVwTooMfdfuU2lYB")\nprint(response)'

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [93]:
assistant = client.beta.assistants.create(
  name=name + " + Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [131]:
file_reference

FileObject(id='file-RiqjOiU8HHACTiJMyYFfWkrU', bytes=406660, created_at=1713069007, filename='2104.05314.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [95]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [99]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [100]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [101]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_aGs2G5hVPx7ejj55Dfj95iUm'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_KJYiOSjuAuc9RJv4UDIczKgI', code_interpreter=CodeInterpreter(input="# Read the file as bytes to understand its content\r\nwith open(file_path, 'rb') as file:\r\n    file_bytes = file.read()\r\n\r\n# Display the first 50 bytes of the file\r\nfile_bytes[:50]", outputs=[CodeInterpreterOutputLogs(logs="b'%PDF-1.4\\r%\\xe2\\xe3\\xcf\\xd3\\r\\n500 0 obj\\r<</Linearized 1/L 406660'", type='logs')]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_DEnZwtCfUhBAsUxN9wkfgCP2'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_mHuOahusj6v1XYFlxVgiD5XM', code_interpreter=CodeInterpreter(input="# Let's read the first few lines of the file to understand its content\r\nwith open(file_path, 'r') as

In [129]:
print(messages.data)
for i in reversed(range(len(messages.data))):
    print(messages.data[i].content[0].text.value)

[Message(id='msg_aGs2G5hVPx7ejj55Dfj95iUm', assistant_id='asst_nlEfM5YuLVwTooMfdfuU2lYB', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The uploaded file appears to be a PDF file based on the content read. The presence of `%PDF-1.4` at the beginning of the file indicates that it is a PDF document. If you have any specific questions or tasks related to this PDF file or if you need assistance with anything else, please feel free to let me know!'), type='text')], created_at=1713105800, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_lABqxQpzoRzvXrUrDiM9f3IX', status=None, thread_id='thread_SRcJDbGVih5YK8cTauOZzrzH'), Message(id='msg_DEnZwtCfUhBAsUxN9wkfgCP2', assistant_id='asst_nlEfM5YuLVwTooMfdfuU2lYB', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='It seems that there was an error in reading the file directly as it might not be a text file. Let me 

In [132]:
file_reference.id

'file-RiqjOiU8HHACTiJMyYFfWkrU'

In [ ]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [138]:
!pip install -qU duckduckgo_search

In [141]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [142]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

"Close. The official 2023 - 2024 roster of the Winnipeg Jets, including position, height, weight, date of birth, age, and birth place. \nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ... \nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ... \nWINNIPEG — The pride in Adam Lowry's voice was evident after being named captain of the Winnipeg Jets on Tuesday. Lowry is the third Jets captain since the team moved from Atlanta in 2011. After going without a captain for the 2022-23 season, Winnipeg chose the rugged centre over alternate captains Josh Morrissey and Mark Scheifele to succeed ... \nAdam Lowry was named captain of the Winnipeg Jets on Tuesday. The 30-year-old forward was selected by the Jets in 

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [144]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}
ddg_function

{'name': 'duckduckgo_search',
 'description': 'Answer non-technical questions. ',
 'parameters': {'type': 'object',
  'properties': {'query': {'type:': 'string',
    'description': "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"}},
  'required': ['query']}}

####❓ Question

Why does the description key-value pair matter?

***ANSWER***
 - Description helps model to decide when/which tool to use during function calling, it helps model to take action based on reasoning (desc helps here).

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [145]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)
assistant

Assistant(id='asst_qaPLg2lbWOW3ROjVCq8yUYu0', created_at=1713108056, description=None, file_ids=[], instructions="You are an expert assistant answering technical questions on machine learning and deep learning subject.\nEnsure that your response is unbiased and generic, you will be 'AWARDED' for giving really good clarity and correct answers.", metadata={}, model='gpt-3.5-turbo', name='ML & DL Assistant + Function Calling API', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='duckduckgo_search', description='Answer non-technical questions. ', parameters={'type': 'object', 'properties': {'query': {'type:': 'string', 'description': "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"}}, 'required': ['query']}), type='function')])

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [146]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

***ANSWER***

- This functions helps to create new thread, create and add messages to thread, helps thread to run, get messages from thread, also function helps to use tools using function calling.
- Functions return thread id.

In [147]:
assistant

Assistant(id='asst_qaPLg2lbWOW3ROjVCq8yUYu0', created_at=1713108056, description=None, file_ids=[], instructions="You are an expert assistant answering technical questions on machine learning and deep learning subject.\nEnsure that your response is unbiased and generic, you will be 'AWARDED' for giving really good clarity and correct answers.", metadata={}, model='gpt-3.5-turbo', name='ML & DL Assistant + Function Calling API', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='duckduckgo_search', description='Answer non-technical questions. ', parameters={'type': 'object', 'properties': {'query': {'type:': 'string', 'description': "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"}}, 'required': ['query']}), type='function')])

In [148]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_OJve0R3nbfCxwVODnwitm9Im', 'output': 'Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ... \nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ... \nNational team. Canada. NHL Draft. 67th overall, 2011. Winnipeg Jets. Playing career. 2013-present. Adam Lowry (born March 29, 1993) is an American-born Canadian professional ice hockey centre and captain of the Winnipeg Jets of the National Hockey League (NHL). \nThe Winnipeg Jets have a new leader, one year after stripping the C from Blake Wheeler and deciding to play without a captain. Adam Lowry, who has been a J

'thread_v16eZOo0rCY161hZ5uIjCAJh'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [152]:
file_reference

FileObject(id='file-RiqjOiU8HHACTiJMyYFfWkrU', bytes=406660, created_at=1713069007, filename='2104.05314.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [153]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [156]:
client.beta.assistants.list().data

[Assistant(id='asst_McpTkpcakxbTjjPldMRudLFG', created_at=1713111965, description=None, file_ids=['file-RiqjOiU8HHACTiJMyYFfWkrU'], instructions="You are an expert assistant answering technical questions on machine learning and deep learning subject.\nEnsure that your response is unbiased and generic, you will be 'AWARDED' for giving really good clarity and correct answers.", metadata={}, model='gpt-3.5-turbo', name='ML & DL Assistant + All Tools', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter'), RetrievalTool(type='retrieval'), FunctionTool(function=FunctionDefinition(name='duckduckgo_search', description='Answer non-technical questions. ', parameters={'type': 'object', 'properties': {'query': {'type:': 'string', 'description': "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"}}, 'required': ['query']}), type='function')]),
 Assistant(id='asst_qaPLg2lbWOW3ROjVCq8yUYu0', created_at=1713108056, description=None, file_id

In [157]:
assistant.id

'asst_McpTkpcakxbTjjPldMRudLFG'

In [158]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The current Captain of the Winnipeg Jets is Blake Wheeler【7:0†source】.
user: Who is the current Captain of the Winnipeg Jets?


'thread_uX0Iforfl1jvZe0hykhMRn91'

In [159]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The authors of the supplied file are:

1. Christian Janiesch
2. Patrick Zschech
3. Kai Heinrich

They are affiliated with different academic institutions as follows:
- Christian Janiesch is from the Faculty of Business Management & Economics, University of Würzburg, Germany.
- Patrick Zschech is from the Institute of Information Systems, Friedrich-Alexander University Erlangen-Nürnberg, Germany.
- Kai Heinrich is from the Faculty of Economics and Management, Otto-von-Guericke-Universität Magdeburg, Germany【5†source】.
user: Who is the author of the supplied file?


'thread_wak7QimvBHnAcHCxzCR8VSOD'

In [160]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The provided file is 406,660 bytes in size.
user: How many bytes is the provided file?


'thread_FWv5SfGRNboWeAVDGY42LlPN'

In [164]:
use_assistant("Who is the current Captain of the Winnipeg Jets search internet?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_jh444hquVjdmsFWpUpHO7cED', 'output': "Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ... \nThe Winnipeg Jets have a new leader, one year after stripping the C from Blake Wheeler and deciding to play without a captain. Adam Lowry, who has been a Jet since 2011 when he was drafted 67th ... \nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ... \nThe Winnipeg Jets have officially made the decision on which player will wear the captain's 'C' for the 2023-24 season, and hopefully onward. That player is 30-year-old centre Adam Lowry.

'thread_NVUQrFY6GCTLz7U9U1ZUUf8c'

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

***ANSWER***:
- Agent/Assisstant will deciode which tool to use through reasoning based on the tools(description) we provide as function.
- Assisstant know's what tools it has access to, which we will give when creating Assisstant's. Ig user asks query to give code it knows to code_interpreter, if user asks question from the files it uses retrieval, if user asks any question which requires custom tool which we pass as fucntion, then asssisstant uses that tool using function calling.

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [179]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )
  #print(completed_response)
  return completed_response

In [180]:
query = "what is ML?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: Machine learning (ML) is a branch of artificial intelligence (AI) that focuses on developing algorithms and techniques that enable computers to learn from and make decisions or predictions based on data. In essence, machine learning allows computers to learn and improve their performance on a task without being explicitly programmed for that task.

ML algorithms are designed to analyze data, identify patterns, and make decisions or predictions based on these patterns. There are various types of machine learning approaches, including supervised learning, unsupervised learning, semi-supervised learning, reinforcement learning, and more. Each approach is suited for different types of tasks and data.

Supervised learning involves training a model on labeled data, where the algorithm learns to map inputs to outputs. Unsupervised learn

Now we can observe JSON mode in action!

In [181]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)

response = messages.data[0].content[0].text.value
print(response)
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

Machine learning (ML) is a branch of artificial intelligence (AI) that focuses on developing algorithms and techniques that enable computers to learn from and make decisions or predictions based on data. In essence, machine learning allows computers to learn and improve their performance on a task without being explicitly programmed for that task.

ML algorithms are designed to analyze data, identify patterns, and make decisions or predictions based on these patterns. There are various types of machine learning approaches, including supervised learning, unsupervised learning, semi-supervised learning, reinforcement learning, and more. Each approach is suited for different types of tasks and data.

Supervised learning involves training a model on labeled data, where the algorithm learns to map inputs to outputs. Unsupervised learning, on the other hand, deals with unlabeled data, where the algorithm learns to find patterns or structure in the data. Reinforcement learning involves traini

In [184]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [217]:
# Experimentiing for bonus challenge
query = "give code of dhage?, try and if you don'y know say don't know after some retries"

thread_id_for_response = use_assistant(query, assistant.id)
print("*********")
print(thread_id_for_response)
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value

completed_flag = json.loads(is_complete(query, response).choices[0].message.content)
print("########################################################")
print(completed_flag)
print("########################################################")
while completed_flag['completed'] == False:
    thread_id_for_response = use_assistant(query, assistant.id)
    print("*********")
    print(thread_id_for_response)
    messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
    response = messages.data[0].content[0].text.value
    completed_flag = json.loads(is_complete(query, response).choices[0].message.content)
    print("########################################################")
    print("Completed flag check based on repsonse: " + str(completed_flag['completed']))
    print("########################################################")

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: I'm not sure what you mean by "dhage" in this context. Could you please provide more information or clarify your question so I can assist you better?
user: give code of dhage?, try and if you don'y know say don't know after some retries
*********
thread_v6mLSjNpMQF9ZC1ct23HSyeo
ChatCompletion(id='chatcmpl-9DxmP4rNgVEbxx4M7nNtBttRfxrze', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "completed": false\n}', role='assistant', function_call=None, tool_calls=None))], created=1713115085, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=8, prompt_tokens=106, total_tokens=114))
########################################################
{'completed': False}
########################################################
Creating Assistant 


## 🚧SOLUTION🚧

In [226]:
MAX_RETRIES = 5
assistant_id = assistant.id

query = "Give the code of hagedfg?, try and if you don't know say don't know after some retries"

def query_assistant_until_complete(query, assistant_id):
    retries = 0
    while retries < MAX_RETRIES:
        thread_id_for_response = use_assistant(query, assistant_id)
        print("*********")
        print("Thread ID for response:", thread_id_for_response)

        messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
        if not messages.data:
            print("No messages returned.")
            retries += 1
            time.sleep(1)  
            continue
        
        response = messages.data[0].content[0].text.value

        completed_flag = json.loads(is_complete(query, response).choices[0].message.content)
        print("########################################################")
        print("Completed flag:", completed_flag)
        print("########################################################")

        if completed_flag.get('completed', False):
            print("")
            print("Assistant has successfully completed the response.🎉🎉🎉")
            break
        else:
            print("Response not completed, retrying...")
            retries += 1
            time.sleep(2) 

        if retries == MAX_RETRIES:
            print("Max retries reached without a completed response.")
        
    return thread_id_for_response

thread_id_for_response = query_assistant_until_complete(query, assistant_id)


Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: I'm not sure what you mean by "hagedfg?" Could you please provide more context or clarify your question so I can better assist you?
user: Give the code of hagedfg?, try and if you don't know say don't know after some retries
*********
Thread ID for response: thread_SjVGZ2UutUkdxlabu5YM7TEH
ChatCompletion(id='chatcmpl-9DyVLG87nbZWTHJ6vnXzCwOgrRZ1X', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "completed": false\n}', role='assistant', function_call=None, tool_calls=None))], created=1713117871, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=8, prompt_tokens=105, total_tokens=113))
########################################################
Completed flag: {'completed': False}
#################################################

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [175]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

In [176]:
file_deletion_status

FileDeleteResponse(id='file-RiqjOiU8HHACTiJMyYFfWkrU', deleted=True, object='assistant.file.deleted')